In [1]:
# Actually you didn't remove bambi and arviz and formulae because you installed them from
# jupyter directly with magic % => solve this problem (if they get imported they are still installed)
# import arviz as az
# import bambi as bmb
# import formulae
# from formulaic import formula

In [2]:
# A script to automate models generation, models performances
# and graphical representations for psychological research. It
# generates a JSON report and takes data from it for an
# intelligent categorization of the database variablels.

In [3]:
# You must have R installed sistem-wide at '/usr/lib/R', the default
# installation folder in Linux operating systems. Otherwise modify
# the path of the R_HOME environment variable.
# It is recommended to install R from terminal because doing it from jupyter
# may cause problems for password typing and admin permissions when
# installation commands are run.
# It is recommended to install all python packages in a virtual environment.
# If you're using a python venv to install packages, install them with
# pip with the -m option. This with install the packages in the
# virtual environment.
# To install them from Jupyter Notebook run "!python3 -m pip install rpy2"
# in a cell.

# Install pillow from Jupyter cell with "%pip install pillow"
# Install ipympl from Jupyter cell with "%pip install ipympl"

# Install LMER packages (THIS TAKES ABOUT 3~5 minutes)
# packnames = ('lme4', 'lmerTest', 'emmeans', 'geepack', 'performance', 'graphics')
# utils = rpy2.robjects.packages.importr("utils")
# utils.chooseCRANmirror(ind=1)
# utils.install_packages(rpy2.robjects.vectors.StrVector(packnames))

In [5]:
# Importing Python packages
import os
import rpy2
import pandas as pd

# Set env variable R_HOME through python. Change the path according to your platform/OS
# and your filesystem.
os.environ['R_HOME'] = '/usr/lib/R'

# Enable cell magic for rpy2 interface.
# %R and %%R are the line and cell magics, respectively.
%load_ext rpy2.ipython

# Loading R packages
graphics = rpy2.robjects.packages.importr('graphics') # Equivalent to "library(graphics)"
lme4 = rpy2.robjects.packages.importr('lme4')
lmerTest = rpy2.robjects.packages.importr('lmerTest')
performance = rpy2.robjects.packages.importr('performance')
graphics = rpy2.robjects.packages.importr('graphics')

# Importing my modules
from data_cleaner import clean_dataframe
import ydata_profiling_generator
from vars_conversion import load_yprofiling_report, build_dtypes_dict, convert_dtypes, represent_dtype_changelog, convert_categoricals_to_strings
from print_models_amount import models_amount_msg
import models_generator_complex_predictors_selection
import models_features
import models_comparison
import r_warnings
import r_graphics

In [6]:
# Load database and build pandas DataFrame
df = pd.read_csv("./data/Titanic-Dataset.csv")

In [7]:
# Generate ydata-profiling report and store it in same
# folder where this script is stored.
ydata_profiling_generator.generate_profiling_report(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/franaz/coding/python/.venv/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'S'')
  warnings.warn(
/home/franaz/coding/python/.venv/lib/python3.10/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/home/franaz/coding/python/.venv/lib/python3.10/site-packages/ydata_profiling/model/missing.py:78: UserWarning: There was an attempt to generate the Heatmap missing values diagrams, but this failed.
To hide this warning, di

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render JSON:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# Clean the dataframe and print shape difference
df = clean_dataframe(df)

# Please note:
# For semplicity and function reuse I've decided to keep
# df and not cleaned_df. After this point we will then keep
# df, but it is actually the cleaned df instead of the original one.

Original df shape: (891, 12)
Cleaned df shape: (891, 11)


In [9]:
# Generates the ydata-profiling report in the same
# folder of this script and loads it in this
# environment, allowing ops with the JSON reported data.
# It intelligrently gives his data types to
# the variables in the dataframe, which will
# be useful for vars conversion later.
data_profile = load_yprofiling_report()

In [10]:
# n.b.! Theese dtypes are not the same used by pandas, so we must match
# them before converting the vars of a pandas dataframe.
report_dtypes = build_dtypes_dict(data_profile)

In [11]:
# Create an object that contains the original dtypes
original_dtypes = df.dtypes

In [12]:
# Convert the DataFrame dtypes
# "Not found" columns should be the ones that have been deleted
# after cleaning
df = convert_dtypes(df, report_dtypes)

'Column cabin not found in DataFrame'


In [13]:
represent_dtype_changelog(df, original_dtypes)

-----------------------
Variable conversion log
-----------------------

survived: int64 --> category
pclass: int64 --> category
sex: object --> category
embarked: object --> category


In [14]:
# Converting df from pandas dataframe to R database (but it is not yet in
# the global envirmonment of R).

# Converting data frames back and forth between rpy2 and pandas should
# be largely automated (no need to convert explicitly, it will be done
# on the fly in most rpy2 functions). To convert explicitly, the functions
# are pandas2ri.py2ri() and pandas2ri.ri2py().
rpy2.robjects.pandas2ri.activate()

In [15]:
# Converting category dtypes to strings so that R can operate on them.
# This is necessary since R uses slightly different data type
# than Pandas. Pandas has a data type for categorical data.
df_r = convert_categoricals_to_strings(df)

In [16]:
# Transfer the DataFrame to R, making it accessible as a
# global envirmonment variable from R and callable usable
# from python.
rpy2.robjects.globalenv['df_r'] = rpy2.robjects.pandas2ri.py2rpy(df_r)

In [17]:
# Setting the desired variables for the models.
response_var="age"
predictor_vars=["survived", "sex"]

In [18]:
model_formulas = models_generator_complex_predictors_selection.generate_all_models(df, response_var, predictor_vars)

In [19]:
models_amount_msg(model_formulas)



------------------------------------------------------------------------
Generating and comparing 40 models...
------------------------------------------------------------------------



In [20]:
model_formulas

['age ~ 1',
 'age ~ survived',
 'age ~ sex',
 'age ~ survived + sex',
 'age ~ survived:sex',
 'age ~ survived + sex + survived:sex',
 'age ~ survived + survived:sex',
 'age ~ sex + survived:sex',
 'age ~ 1 + (0 + survived + sex | survived)',
 'age ~ sex + (1 | survived)',
 'age ~ 1 + (sex | sex)',
 'age ~ survived + (0 + survived | survived)',
 'age ~ survived + sex + (1 | survived)',
 'age ~ survived + (survived | survived)',
 'age ~ survived + sex + (survived + sex | survived)',
 'age ~ survived + sex + (1 | sex)',
 'age ~ sex + (sex | survived)',
 'age ~ 1 + (0 + survived | sex)',
 'age ~ 1 + (0 + sex | sex)',
 'age ~ survived + (1 | sex)',
 'age ~ sex + (0 + sex | survived)',
 'age ~ 1 + (sex | survived)',
 'age ~ survived + (survived | sex)',
 'age ~ sex + (sex | sex)',
 'age ~ 1 + (0 + survived + sex | sex)',
 'age ~ 1 + (1 | sex)',
 'age ~ survived + (0 + survived | sex)',
 'age ~ survived + (1 | survived)',
 'age ~ 1 + (0 + sex | survived)',
 'age ~ survived + sex + (0 + surviv

In [21]:
# but probably it generated that warning because your random effects are very small.
# It is likely that one of your variables is not being evaluated as contributing to the
# variance. Most likely, the "culprit" is the random variable (subj in your case). If you look
# at the output of your model (just print ce_model and hit enter, or summary(ce_model)), there
# should be a variable that has zero or very nearly zero variance and standard deviation.
# In lmer, a singular fit could be caused by collinearity in fixed effects, as in any other linear model.
# That would need you to revise your model by removing terms. But in lmer, that (or a "boundary (singular)
# fit" warning) can also be also triggered in quite simple models when a random effect variance 
# is estimated very near zero and (very loosely) the data is not sufficiently informative to drag the
# estimate away from the zero starting value.
# The formal answer is broadly similar either way; drop terms that estimate as zero.
evaluation_results = models_features.compute_models_indexes(df, model_formulas)

Evaluating models:   0%|                                                | 0/4 [00:00<?, ?it/s]R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Random effect variances not available. Returned R2 does not account for random effects.


Evaluating models:  25%|██████████                              | 1/4 [00:01<00:04,  1.54s/it]R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Random effect variances not available. Returned R2 does not account for random effects.


R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



...

R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Random effect variances not available. Returned R2 does not account for random effects.


R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Random effect variances not available. Returned R2 does not account for random effects.


Evaluating models: 100%|████████████████████████████████████████| 4/4 [00:11<00:00,  2.82s/it]


In [22]:
# Perform weighted evaluation
best_models = models_comparison.weighted_evaluation(evaluation_results['non_mixed'], evaluation_results['mixed'])

Best Non-Mixed Model: {'formula': 'age ~ survived', 'aic': 11640.117184131024, 'bic': 11649.701872985966, 'r_squared': 0.009479065631399664, 'adj_r_squared': 0.00836486885483212}
Best Mixed Model: {'formula': 'age ~ survived + (0 + survived | sex)', 'aic': 11636.138616468017, 'bic': 11664.892683032842, 'conditional_r_squared': 0.009468515841869921}


In [23]:
# Print R warnings
r_warnings.print_r_warnings()

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



In [24]:
# '%matplotlib inline' causes the plots to appear right below the cell.
# '%matplotlib notebook' magic command does not only plots the graph below the cell; it 
# also plots the graph in an interactive window.
%matplotlib inline
%matplotlib ipympl

# Plotting the two best models
r_graphics.plot_best_models_diagnostics(best_models, df_r)

R[write to console]: boundary (singular) fit: see help('isSingular')



In [25]:
# Print R warnings
r_warnings.print_r_warnings()

Warning messages:
1: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
2: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
3: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
4: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
5: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
6: In (function (package, help, pos = 2, lib.loc = NULL,  ... :
  library ‘/usr/lib/R/site-library’ contains no packages
7: Can't compute random effect variances. Some variance components equal
  zero. Your model may suffer from singularity (see `?lme4::isSingular`
  and `?performance::check_singularity`).
  Solution: Respecify random structure! You may also decrease the
  `